In [1]:
import sys 
sys.path.append('../Nets')
import sys
sys.setrecursionlimit(10000)
from multiprocessing import Pool
from PIL import Image
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm
from functools import partial
from keras.optimizers import RMSprop
from glob import glob
import shutil
from os.path import join, basename
from ResNet import *
from scipy.ndimage import zoom
from scipy.ndimage.interpolation import rotate
import os
from keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
%pylab inline

Using Theano backend.


Populating the interactive namespace from numpy and matplotlib


In [2]:
PATH = {
    'DATA': '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/TRAIN',
    'DATA_OUT': '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/TRAIN_OUT',
    'CSV': '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/CSV',
    'LABELS': '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/CSV/stage1_labels.csv',

    'LUNA': '/fasthome/a.dobrenkii/LUNA',
    'LUNA_DATA': '/fasthome/a.dobrenkii/LUNA/DATA',
    'LUNA_SOBEL': '/fasthome/a.dobrenkii/LUNA/SOBEL_IMG',
    'LUNA_LUNGS': '/fasthome/a.dobrenkii/LUNA/LUNGS_IMG',
    'LUNA_MASKS': '/fasthome/a.dobrenkii/LUNA/MASKS',
    'LUNA_CSV': '/fasthome/a.dobrenkii/LUNA/CSVFILES',
    'LUNA_PRED': '/fasthome/a.dobrenkii/LUNA/PRED',

    'WEIGHTS': '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/WEIGHTS',
    'CHECKPOINTS': '/home/a.dobrenkii/Projects/Kaggle/DataScienceBowl2K17/data/'
}

# annotations = pd.read_csv(join(PATH['LUNA_CSV'], 'annotations.csv'))
# candidates = pd.read_csv(join(PATH['LUNA_CSV'], 'candidates.csv'))

In [3]:
model, bottleneck = ResNet50(weights='imagenet', out=2)
bottleneck = load_weighs(bottleneck, False)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

K.image_dim_ordering: th


In [4]:
dim = 224
nb_epoch = 10
CPU = 10
chanels = 3
train_test = .3
shape = (3, 224, 224)

In [5]:
def crop_and_not_rotate(segmenteds):
    x, y, z = where(segmenteds)
    if len(x):
        segmenteds = segmenteds[x.min():x.max(), y.min():y.max(), z.min():z.max()]
    else:
        segmenteds = array([])
    return segmenteds

In [6]:
def load_and_augment(lung_path, mask_path, shape, angle=15):
    lung = load(lung_path)
    mask = load(mask_path)
    
    
    position = random.randint(0, lung.shape[0] - shape[0])
    lung = lung[position: position + shape[0]]
    mask = (mask[position: position + shape[0]] == 2).sum() != 0
    
    
    lung = rotate(lung, random.randint(-angle, angle), axes=[1,2])
  
    x, y, z = where(lung)
    if len(x):
        lung = lung[:, y.min():y.max(), z.min():z.max()]
    
    if 'left' in lung_path.lower():
        lung = fliplr(lung)
    
    
    mn = lung.min()
    mx = lung.max()
    if (mx - mn) != 0:
        lung = (lung - mn) / (mx - mn)
    else:
        lung[:, :] = 0
      
    new_shape = list(shape)
    ln_axis = argmax(lung.shape[1:])
    sh_axis = argmin(lung.shape[1:])    
    new_shape[sh_axis + 1] = int(lung.shape[sh_axis + 1] * (shape[sh_axis + 1] 
                                 / lung.shape[ln_axis + 1]))
    factor = [1] + [n / o for o, n in zip(lung.shape[1:], new_shape[1:])]
    lung = zoom(lung, factor, order=2)
    if lung.shape[sh_axis + 1] != shape[sh_axis + 1]:
        top = (shape[1] - lung.shape[sh_axis + 1])
        bottom = random.randint(0, top)
        
        pad_values = [[top - bottom, bottom]  
                      if i == sh_axis + 1 
                      else [0, 0] 
                      for i in range(len(shape))]
        
        lung = pad(lung, pad_values, 'constant')
        
    if mask == 0:
        mask = [1, 0]
    else:
        mask = [0, 1]
    return lung, mask


In [7]:
def augment_slice(lung, shape, angle=15):
    
    lung = rotate(lung, random.randint(-angle, angle), axes=[1,2])
  
    x, y, z = where(lung)
    if len(x):
        lung = lung[:, y.min():y.max(), z.min():z.max()]
    
    mn = lung.min()
    mx = lung.max()
    if (mx - mn) != 0:
        lung = (lung - mn) / (mx - mn)
    else:
        lung[:, :] = 0
      
    new_shape = list(shape)
    ln_axis = argmax(lung.shape[1:])
    sh_axis = argmin(lung.shape[1:])    
    new_shape[sh_axis + 1] = int(lung.shape[sh_axis + 1] * (shape[sh_axis + 1] 
                                 / lung.shape[ln_axis + 1]))
    factor = [1] + [n / o for o, n in zip(lung.shape[1:], new_shape[1:])]
    lung = zoom(lung, factor, order=2)
    if lung.shape[sh_axis + 1] != shape[sh_axis + 1]:
        top = (shape[1] - lung.shape[sh_axis + 1])
        bottom = random.randint(0, top)
        
        pad_values = [[top - bottom, bottom]  
                      if i == sh_axis + 1 
                      else [0, 0] 
                      for i in range(len(shape))]
        
        lung = pad(lung, pad_values, 'constant')
    return lung

In [8]:
def load_and_augment(lung_path, mask_path, shape, angle=15):
    lung = load(lung_path)
    mask = load(mask_path)
    if 'left' in lung_path.lower():
        lung = fliplr(lung)
    lung = [lung[i: i + shape[0]] for i in range(lung.shape[0] - shape[0])]
    mask = [[1, 0] if (mask[i: i + shape[0]] == 2).sum() != 0 else [0, 1] 
            for i in range(mask.shape[0] - shape[0])]
    augment = partial(augment_slice, shape=shape, angle=angle)
    with Pool(CPU) as pool:
        lungs = pool.map(augment, lung)
    return lungs, mask


In [9]:
def batch_generator(lung_paths, mask_paths, batch_size):
    number_of_batches = ceil(len(lung_paths) / batch_size)
    counter = 0
    
    combined = list(zip(lung_paths, mask_paths))
    random.shuffle(combined)
    lung_paths, mask_paths = zip(*combined)
    
    while True:
        batch_files = (lung_paths[batch_size * counter:batch_size * (counter + 1)],
                       mask_paths[batch_size * counter:batch_size * (counter + 1)])
        
        lungs_list = list()
        mask_list = list()
        for lung, mask in zip(batch_files[0], batch_files[1]):
            image, mask = load_and_augment(lung, mask, (chanels, dim, dim))
                
            lungs_list += image
            mask_list += mask
            
        combined = list(zip(lungs_list, mask_list))
        random.shuffle(combined)
        lungs_list, mask_list = zip(*combined)
        counter += 1
        yield array(lungs_list), array(mask_list)
        
        if counter == number_of_batches:
            combined = list(zip(lung_paths, mask_paths))
            random.shuffle(combined)
            lung_paths, mask_paths = zip(*combined)
            counter = 0

In [10]:
def mini_batch_generator(lung_paths, mask_paths, batch_size):
    number_of_batches = ceil(len(lung_paths) / batch_size)
    counter = 0
    
    combined = list(zip(lung_paths, mask_paths))
    random.shuffle(combined)
    lung_paths, mask_paths = zip(*combined)
    
    while True:
        batch_files = (lung_paths[batch_size * counter:batch_size * (counter + 1)],
                       mask_paths[batch_size * counter:batch_size * (counter + 1)])
        
        lungs_list = list()
        mask_list = list()
        for lung, mask in zip(batch_files[0], batch_files[1]):
            image, mask = load_and_augment(lung, mask, (chanels, dim, dim))
                
            lungs_list.append(image)
            mask_list.append(mask)
            
        counter += 1
        yield array(lungs_list), array(mask_list)
        
        if counter == number_of_batches:
            combined = list(zip(lung_paths, mask_paths))
            random.shuffle(combined)
            lung_paths, mask_paths = zip(*combined)
            counter = 0

In [ ]:
paths = list()
lung_paths = glob(join(PATH['LUNA_LUNGS'], '*'))
mask_paths = list()
for lung in tqdm(lung_paths):
    if 'left' in lung:
        side = 'left'
    else:
        side = 'right'
    pid = basename(lung).split('lungs')[0] 
    mask = glob(join(PATH['LUNA_MASKS'], '*' + pid + '*' + side + '*'))
    if len(mask) == 1:
        paths.append([lung, mask[0]])

shuffle(paths)

In [12]:
save(join(PATH['LUNA'], 'paths'), asarray(paths))

In [11]:
paths = load(join(PATH['LUNA'], 'paths.npy'))

In [13]:
lung_paths, mask_paths = zip(*paths)


train_lung, train_mask = zip(*paths[int(len(paths) * train_test):])
valid_lung, valid_mask = zip(*paths[:int(len(paths) * train_test)])

In [ ]:
train_generator = batch_generator(train_lung, train_mask, 32)
valid_generator = batch_generator(valid_lung, valid_mask, 5)
for e in range(1):
    print("Epoch %d / %d" % (e, nb_epoch))
    valid = next(valid_generator)
    for X_train, Y_train in train_generator:
        model.fit(X_train, 
                  Y_train, 
                  batch_size=16, 
                  nb_epoch=1, 
                  validation_data=valid,
                  shuffle=True)
    model.save_weights("model.h5")

# for e in range(nb_epoch):
#     print("epoch %d" % e)
#     for X_train, Y_train in batch_generator(lung_paths, mask_paths, 2):
#         model.train(X_batch, Y_batch)
#     model.save_weights("model" + e + ".h5")

Epoch 0 / 10
Train on 11098 samples, validate on 1581 samples
Epoch 1/1
 3632/11098 [========>.....................] - ETA: 6337s - loss: 0.1992 - acc: 0.9499

In [ ]:
checkpoint = ModelCheckpoint(filepath=PATH['WEIGHTS'], 
                             verbose=1, 
                             save_best_only=True)


model.fit_generator(train_generator, 
                    samples_per_epoch=2000, 
                    nb_epoch=20, 
                    callbacks=[checkpoint], 
                    validation_data=valid_generator, 
                    class_weight=None, 
                    nb_val_samples=532,
                    nb_worker=1)